# Pengembangan titik tambang OilyGiant

## Konten<a id='back'></a>

* [Pendahuluan](#intro)
* [Tahap 1. Inisialisasi data](#inisial)
* [Tahap 2. Mempersiapkan data data](#prepare)
* [Tahap 3. Melatih dan menguji model](#traintest)
* [Tahap 4. Menghitung laba](#laba)
* [Tahap 5. Estimasi Margin Profit](#profit)
* [Tahap 6. Hitung Resiko dan keuntungan](#risk)
* [Kesimpulan Umum](#conclusion)
    * [Temuan](#end)
    * [Rekomendasi](#recom)


## Pendahuluan<a id='intro'></a>
OilyGiant adalah perusahaan tambang dan sedang mencari tempat terbaik untuk mengembangkan 200 titik sumur minyak baru. Untuk menyelesaikan tugas ini, langkah-langkah yang harus dilakukan adalah sebagai berikut:
1. Baca file dengan parameter yang dikumpulkan dari sumur-sumur minyak di wilayah terpilih: kualitas minyak dan volume cadangannya;
2. Buat sebuah model untuk memprediksi volume cadangan di sumur-sumur baru;
3. Pilih sumur minyak yang memiliki nilai estimasi tertinggi;
4. Pilih wilayah dengan total keuntungan tertinggi dari sumur minyak yang dipilih,
5. Ada data sampel minyak dari tiga wilayah dan parameter setiap sumur minyak di wilayah tersebut sudah diketahui. Buat sebuah model yang akan membantu memilih wilayah dengan margin keuntungan tertinggi. Gunakan teknik bootstrapping untuk menganalisis potensi keuntungan dan risiko.

Instruksi Proyek
1. Unduh dan siapkan datanya. Jelaskan prosedur yang kamu lakukan.
2. Latih dan uji model untuk setiap wilayah di `'geo_data_0.csv'`:
    * Pisahkan data menjadi training set dan validation set dengan rasio 75:25.
    * Latih model dan buat prediksi untuk validation set.
    * Simpan prediksi dan jawaban yang benar untuk validation set.
    * Tampilkan prediksi volume rata-rata pemesanan dan RMSE dari model.
    * Analisis hasilnya.
    * Lakukan dan jalankan langkah 1-5 untuk file `'geo_data_1.csv'` dan `'geo_data_2.csv'`.
3. Lakukan persiapan untuk menghitung laba:
    * Simpan semua nilai yang dibutuhkan untuk perhitungan laba pada variabel terpisah.
    * Dengan investasi 100 juta untuk 200 sumur minyak, sebuah sumur minyak rata-rata harus menghasilkan setidaknya 500 ribu USD untuk menghindari kerugian (ini setara dengan 111,1 unit). Bandingkan jumlah ini dengan jumlah rata-rata pemesanan di setiap wilayah.
    * Berikan kesimpulan terkait persiapan yang kamu lakukan dalam langkah penghitungan laba.

4. Buat sebuah fungsi untuk menghitung laba dari kumpulan sumur minyak terpilih dan prediksi model:
    * Tuliskan sebuah fungsi untuk menghitung keuntungan dari sekumpulan sumur minyak terpilih dan prediksi modelnya: Pilih 200 sumur dengan nilai prediksi tertinggi dari masing-masing 3 wilayah (file 'csv').
    * Buat rangkuman tentang target volume pemesanan berdasarkan prediksi ini. Simpan prediksi untuk 200 sumur di masing-masing dari 3 wilayah.
    * Sampaikan kesimpulanmu: sarankan sebuah wilayah yang cocok untuk pengembangan sumur minyak dan berikan alasan atas pilihanmu. 
    
5. Hitung keuntungan untuk volume pemesanan yang diterima.
    * Hitung risiko dan keuntungan untuk setiap wilayah: Dengan menggunakan prediksi yang kamu simpan pada langkah 2 di atas, gunakan teknik bootstrapping dengan 1.000 sampel untuk menemukan distribusi profit.
    * Temukan rata-rata profit, interval kepercayaan 95%, dan risiko kerugian. Kerugian adalah keuntungan negatif, hitung sebagai probabilitas dan kemudian nyatakan sebagai persentase.
    * Sampaikan kesimpulanmu: sarankan sebuah wilayah yang cocok untuk pengembangan sumur minyak dan berikan alasan atas pilihanmu.
    
Deskripsi Data

Data eksplorasi geologi untuk ketiga wilayah disimpan dalam beberapa file:
`geo_data_0.csv`. unduh dataset
`geo_data_1.csv`. unduh dataset
`geo_data_2.csv`. unduh dataset
id — ID unik sumur minyak
f0, f1, f2 — tiga fitur titik wilayah (makna spesifiknya sebenarnya tidak penting, tetapi fitur itu sendiri sangat penting) product — volume cadangan di dalam sumur minyak (1 unit = 1.000 barel).

Kondisi:
1. Hanya regresi linear yang bisa digunakan untuk pelatihan model (sisanya tidak cukup mudah diprediksi).
2. Anggaran untuk pengembangan 200 sumur minyak ini adalah 100 juta dolar.
3. Satu barel minyak mentah menghasilkan pendapatan sebesar 4,5 dolar. Nah, pendapatan dari satu unit produk adalah $4.500 (volume cadangan ditulis dalam ribuan barel).

Setelah kamu menganalisis risiko yang ada, pertahankan hanya wilayah yang memiliki risiko kerugian kurang dari 2,5%. Di antara semua yang sesuai dengan kriteria, kamu harus memilih wilayah dengan keuntungan rata-rata tertinggi.

## Inisialisasi Data<a id='inisial'></a>

memasukkan semua library yang dibutuhkan:

In [1]:
# memuat library yang dibutuhkan
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# set tema seaborn
sns.set_theme(palette='bright')

## Mempersiapkan data<a id='prepare'></a>

### Dataset `'/datasets/geo_data_0.csv'`

In [2]:
# Mengunduh dan memeriksa data '/datasets/geo_data_0.csv'
geo0 = pd.read_csv('/datasets/geo_data_0.csv')
geo0.head(10)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


In [3]:
#memeriksa info umum
geo0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
# mengecek nilai yang hilang pada dataset
geo0.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [5]:
# mengecek duplikasi pada dataset
geo0.duplicated().sum()

0

dataset terdiri dari 5 kolom, `id`, `f0`, `f1`, `f2`, dan `product` dan 100000 data, tipe data sudah benar, tidak ada nilai hilang dan tidak ada duplikasi data.
* `id` — ID unik sumur minyak
* `f0`, `f1`, `f2` — tiga fitur titik wilayah.
* `product` — volume cadangan di dalam sumur minyak (1 unit = 1.000 barel).


In [6]:
#informasi umum tambahan
geo0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


### Dataset `'/datasets/geo_data_1.csv'`

In [7]:
# Mengunduh dan memeriksa data '/datasets/geo_data_1.csv'
geo1 = pd.read_csv('/datasets/geo_data_1.csv')
geo1.head(10)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


In [8]:
#memeriksa info umum
geo1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [9]:
# mengecek nilai yang hilang pada dataset
geo1.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [10]:
# mengecek duplikasi pada dataset
geo1.duplicated().sum()

0

dataset terdiri dari 5 kolom, `id`, `f0`, `f1`, `f2`, dan `product` dan 100000 data, tipe data sudah benar, tidak ada nilai hilang dan tidak ada duplikasi data.
* `id` — ID unik sumur minyak
* `f0`, `f1`, `f2` — tiga fitur titik wilayah.
* `product` — volume cadangan di dalam sumur minyak (1 unit = 1.000 barel).


In [11]:
#informasi umum tambahan
geo1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


### Dataset `'/datasets/geo_data_2.csv'`

In [12]:
# Mengunduh dan memeriksa data '/datasets/geo_data_2.csv'
geo2 = pd.read_csv('/datasets/geo_data_2.csv')
geo2.head(10)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


In [13]:
#memeriksa info umum
geo2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [14]:
# mengecek nilai yang hilang pada dataset
geo2.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [15]:
# mengecek duplikasi pada dataset
geo2.duplicated().sum()

0

dataset terdiri dari 5 kolom, `id`, `f0`, `f1`, `f2`, dan `product` dan 100000 data, tipe data sudah benar, tidak ada nilai hilang dan tidak ada duplikasi data.
* `id` — ID unik sumur minyak
* `f0`, `f1`, `f2` — tiga fitur titik wilayah.
* `product` — volume cadangan di dalam sumur minyak (1 unit = 1.000 barel).


In [16]:
#informasi umum tambahan
geo2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


**Kesimpulan:**
ketiga dataset terdiri dari 5 kolom, `id`, `f0`, `f1`, `f2`, dan `product` dan 100000 data, tipe data sudah benar, tidak ada nilai hilang dan tidak ada duplikasi data. jadi tidak perlu penanganan lanjutan dan bisa langsung ke proses selanjutnya.

## Melatih dan menguji model<a id='traintest'></a>

### Memisahkan dataset

dataset akan dipisahkan menjadi `75:25` antara `training set` dan `validation set`

In [17]:
#fungsi pemisahan dataset
def prepare_datasets(geo_df):
    features = geo_df.drop(['product', 'id'], axis=1)
    target = geo_df['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
    return features_train, features_valid, target_train, target_valid

fungsi ini akan membantu untuk memisahkan dataset selanjutnya

### Model `geo0`

In [18]:
# memisahkan dataset geo0
features_train_geo0, features_valid_geo0, target_train_geo0, target_valid_geo0 = prepare_datasets(geo0)

# Menampilkan bentuk dari masing-masing set untuk geo0
print('Training set (Geo0):', features_train_geo0.shape, target_train_geo0.shape)
print('Validation set (Geo0):', features_valid_geo0.shape, target_valid_geo0.shape)

Training set (Geo0): (75000, 3) (75000,)
Validation set (Geo0): (25000, 3) (25000,)


dataset `geo0` sudah dibagi menjadi `75:25` antara `training set` dan `validation set`

In [19]:
# membuat fungsi untuk menghitung rata-rata prediksi volume minyak beserta RMSE model
def oil_pred(feature_train,feature_valid,target_train,target_valid):
    # membuat model regresi linier
    model = LinearRegression()
    # melatih model menggunakan dataset training
    model.fit(feature_train,target_train)
    # memprediksi rata-rata volume minyak
    vol = model.predict(feature_valid)
    # menghitung RMSE
    rmse = mean_squared_error(target_valid,vol)**0.5
    # mengembalikan nilai volume minyak dan nilai RMSE
    return vol, rmse

In [20]:
# menghitung estimasi prediksi rata-rata volume minyak dan nilai RMSEnya untuk dataset geo0
vol_0, rmse_0 = oil_pred(features_train_geo0, features_valid_geo0, target_train_geo0, target_valid_geo0)
print('Rata-rata volume minyak -->', round(vol_0.mean(),2),'ribu barel')
print('RMSE -->', round(rmse_0,2),'ribu barel')

Rata-rata volume minyak --> 92.59 ribu barel
RMSE --> 37.58 ribu barel


Rata-rata volume minyak pada wilayah `geo0` adalah `92.59` ribu barel dengan `RSME` `37.58` ribu barel

### Model `geo1`

In [21]:
# memisahkan dataset geo1
features_train_geo1, features_valid_geo1, target_train_geo1, target_valid_geo1 = prepare_datasets(geo1)

# Menampilkan bentuk dari masing-masing set untuk geo0
print('Training set (Geo1):', features_train_geo1.shape, target_train_geo1.shape)
print('Validation set (Geo1):', features_valid_geo1.shape, target_valid_geo1.shape)

Training set (Geo1): (75000, 3) (75000,)
Validation set (Geo1): (25000, 3) (25000,)


dataset `geo1` sudah dibagi menjadi `75:25` antara `training set` dan `validation set`

In [22]:
# menghitung estimasi prediksi rata-rata volume minyak dan nilai RMSEnya untuk dataset geo1
vol_1, rmse_1 = oil_pred(features_train_geo1, features_valid_geo1, target_train_geo1, target_valid_geo1)
print('Rata-rata volume minyak -->', round(vol_1.mean(),2),'ribu barel')
print('RMSE -->', round(rmse_1,2),'ribu barel')

Rata-rata volume minyak --> 68.73 ribu barel
RMSE --> 0.89 ribu barel


Rata-rata volume minyak pada wilayah `geo1` adalah `68.73` ribu barel dengan `RSME` `0.89` ribu barel. wilayah ini memberikan nilai `RSME` terbaik dan terendah dibandingkan dengan wilayah lain.

### Model `geo2`

In [23]:
# memisahkan dataset geo2
features_train_geo2, features_valid_geo2, target_train_geo2, target_valid_geo2 = prepare_datasets(geo2)

# Menampilkan bentuk dari masing-masing set untuk geo0
print('Training set (Geo2):', features_train_geo2.shape, target_train_geo2.shape)
print('Validation set (Geo2):', features_valid_geo2.shape, target_valid_geo2.shape)

Training set (Geo2): (75000, 3) (75000,)
Validation set (Geo2): (25000, 3) (25000,)


dataset `geo2` sudah dibagi menjadi `75:25` antara `training set` dan `validation set`

In [24]:
# menghitung estimasi prediksi rata-rata volume minyak dan nilai RMSEnya untuk dataset geo2
vol_2, rmse_2 = oil_pred(features_train_geo2, features_valid_geo2, target_train_geo2, target_valid_geo2)
print('Rata-rata volume minyak -->', round(vol_2.mean(),2),'ribu barel')
print('RMSE -->', round(rmse_2,2),'ribu barel')

Rata-rata volume minyak --> 94.97 ribu barel
RMSE --> 40.03 ribu barel


Rata-rata volume minyak pada wilayah `geo2` adalah `94.97` ribu barel dengan `RSME` `40.03` ribu barel. ini adalah wilayah  dengan rata-rata volume minyak tertinggi dibandingkan dengan wilayah lainnya. akan tetapi nilai `RSME` nya juga paling tinggi.

**Kesimpulan:**
wilayah `geo2` memiliki rata-rata volume minyak terbanyak dan `RSME` tertinggi berarti akurasi wilayah ini cukup rendah walaupunmenghasilkan produksi yang banyak. akan tetapi wilayah `geo1` memiliki rata-rata volume terendah dan `RSME` terendah pula, berarti tingkat akurasinya lebih baik dibandingkan dengan `geo2`

## Menghitung Laba<a id='laba'></a>

Berikut ini merupakan variabel penting dalam bisnis `OilyGiant` yaitu modal `investasi`, `jumlah target lokasi titik`, `harga minyak per barel`. Kita akan memperjelasnya kedalam beberapa poin:
* Berapa produksi dalam ribuan barel tiap sumur (titik) agar tidak mengalami kerugian?
* Membandingkan revenue tanpa kerugian dan revenue dari masing-masing wilayah.

In [25]:
# beberapa rangkuman data bisnis
# jumlah lokasi point terbaik
n_best = 200

# modal investasi (USD)
invest = 100000000

# harga per barel (USD/barel)
price_vol = 4.5

# harga per unit produk (USD/1000barel)
price_prod = 4.5*1000

# mengetahui berapa unit produk yang bisa dihasilkan dari modal investasi untuk 200 lokasi
n_prod = invest/(price_prod*n_best)

print('Besarnya Unit Produk Tanpa Rugi',round(n_prod,2),'ribu barel')

Besarnya Unit Produk Tanpa Rugi 111.11 ribu barel


Besarnya Unit Produk Tanpa Rugi 111.11 ribu barel akan dibandingkan dengan nilai rata-rata volume minyak yang sbelumnya di dapatkan.

In [26]:
# menampilkan kembali perhitungan sebelumnya
print('Besarnya Unit Produk Region-1',round(vol_0.mean(),2),'ribu barel')
print('Besarnya Unit Produk Region-2',round(vol_1.mean(),2),'ribu barel')
print('Besarnya Unit Produk Region-3',round(vol_2.mean(),2),'ribu barel')

Besarnya Unit Produk Region-1 92.59 ribu barel
Besarnya Unit Produk Region-2 68.73 ribu barel
Besarnya Unit Produk Region-3 94.97 ribu barel


In [27]:
# Menghitung rata-rata produksi untuk setiap wilayah
mean_production_geo0 = geo0['product'].mean()
mean_production_geo1 = geo1['product'].mean()
mean_production_geo2 = geo2['product'].mean()

# Menyimpan rata-rata produksi ke dalam variabel
mean_productions = {
    'geo0': mean_production_geo0,
    'geo1': mean_production_geo1,
    'geo2': mean_production_geo2
}
# Membandingkan rata-rata produksi dengan minimum produksi rata-rata per sumur
for region, mean_production in mean_productions.items():
    print(f"Wilayah {region} rata-rata produksi: {mean_production:.2f}")
    if mean_production >= n_prod:
        print(f"Wilayah {region} memenuhi atau melebihi ambang batas produksi minimal ({n_prod:.2f} unit).")
    else:
        print(f"Wilayah {region} tidak memenuhi ambang batas produksi minimal ({n_prod:.2f} unit).")

Wilayah geo0 rata-rata produksi: 92.50
Wilayah geo0 tidak memenuhi ambang batas produksi minimal (111.11 unit).
Wilayah geo1 rata-rata produksi: 68.83
Wilayah geo1 tidak memenuhi ambang batas produksi minimal (111.11 unit).
Wilayah geo2 rata-rata produksi: 95.00
Wilayah geo2 tidak memenuhi ambang batas produksi minimal (111.11 unit).


terlihat bahwa semuanya tidak memenuhi ambang batas produksi minimal yaitu `111.11` unit, dan yang paling mendekati adalah wilayah `geo2`

## Estimasi Margin Profit<a id='profit'></a>

### Estimasi Margin Profit tanpa teknik Bootstrap

In [28]:
# membuat fungsi untuk menghitung margin profit antara prediksi model dan revenue tanpa kerugian
# data dari prediksi model(predict) dan data target validasi(target) untuk n_best=200
def margin_profit(predict,target):
    # membuat series untuk data dari prediksi
    pred = pd.Series(predict)
    # mereset index pada data target agar sesuai dengan index prediksi
    tar = target.reset_index(drop=True)
    # mengurutkan data prediksi dari yang terbesar
    pred.sort_values(ascending=False,inplace=True)
    # mengeluarkan nilai target berdasarkan index prediksi yang sudah diurutkan
    tar = tar[pred.index]
    # menghitung selisih revenue antara model prediksi dan revenue tanpa kerugian
    # margin = [jumlah revenue prediksi] - [jumlah revenue tanpa kerugian]
    # mengambil data terbaik sebanyak `n_best=200`
    margin = price_prod*(tar[:n_best].sum() - n_prod*n_best)

    # mengembalikan margin profit
    return margin

In [29]:
# menghitung margin profit menggunakan 200 titik terbaik untuk tiap dataset
rev_geo0 = margin_profit(vol_0,target_valid_geo0)
rev_geo1 = margin_profit(vol_1,target_valid_geo1)
rev_geo2 = margin_profit(vol_2,target_valid_geo2)

# menampilkan revenue 200 titik lokasi terbaik
print('Margin Profit [Region-1] -->',round(rev_geo0,2),'USD')
print('Margin Profit [Region-2] -->',round(rev_geo1,2),'USD')
print('Margin Profit [Region-3] -->',round(rev_geo2,2),'USD')

Margin Profit [Region-1] --> 33208260.43 USD
Margin Profit [Region-2] --> 24150866.97 USD
Margin Profit [Region-3] --> 27103499.64 USD


Ternyata ketika kita mengambil 200 lokasi terbaik dari dataset kita tidak mengalami kerugian, bahkan menunjukan keuntungan (nilai positif margin). Berbeda dengan perhitungan sebelumnya ketika menggunakan rata-rata unit produk. keuntungan terbaik terdapat pada wilayah `geo0`, `geo2` dan `geo1`

### Margin profit menggunakan teknik Bootstrap

Pembahasanya sudah bagus bahwa kita tidak mengalami kerugian bahkan meningkat. Namun kita apakah bisa memprediksi resiko kerugian dengan menggunakan fungsi sebelumnya? kita tidak tahu apakah mungkin terjadi kerugian atau tidak, kecuali dengan teknik bootstrapping seperti berikut untuk melihat distribusi margin profit untuk masing-masing wilayah. Kita akan membuat `1000` sampel bootsrap dengan pengambilan sebanyak `500` sampel untuk menentukan `200` lokasi.

In [30]:
# jumlah sampel bootstrap
n_boot = 1000

# jumlah sampel point
n_point = 500

# membuat fungsi untuk menghitung margin profit menggunakan teknik bootstrap
def margin_bootsrap(predict,target):
    # tempat penampungan nilai margin untuk sampel bootstrap ke-i
    margin = []
    # mengubah data dari predict menjadi series
    predict = pd.Series(predict)
    # mereset index pada data target
    target = target.reset_index(drop=True)
    # membuat randomstate dari numpy
    random = np.random.RandomState(12345)

    # membuat bootstrap untuk sebanyak `n_boot=1000` untuk menghitung revenue
    for i in range(n_boot):
        # mengambil sampel sebanyak `n_point=500` dari data target
        tar_sample = target.sample(n_point,replace=True,random_state=random)
        # mengambil nilai prediksi yang sesuai dengan indeks target sampel
        predict_sample = predict[tar_sample.index]
        # mereset indeks taget sampel
        tar_sample.reset_index(drop=True,inplace=True)
        # mereset indeks prediksi sampel
        predict_sample.reset_index(drop=True,inplace=True)
        # menghitung besarnya margin profit dan menambahkanya ke tempat penampungan `margin`
        margin.append(margin_profit(predict_sample,tar_sample))

    # mengembalikan nilai margin
    return margin

In [31]:
# menghitung profit margin menggunakan teknik bootstrap untuk tiap dataset
rev_geo0_boot = margin_bootsrap(vol_0,target_valid_geo0)
rev_geo1_boot = margin_bootsrap(vol_1,target_valid_geo1)
rev_geo2_boot = margin_bootsrap(vol_2,target_valid_geo2)

# menampilkan rata-rata margin profit menggunakan bootstrap
print('Rata-rata Profit Margin [Region-1] -->',round(np.mean(rev_geo0_boot),2),'USD')
print('Rata-rata Profit Margin [Region-2] -->',round(np.mean(rev_geo1_boot),2),'USD')
print('Rata-rata Profit Margin [Region-3] -->',round(np.mean(rev_geo2_boot),2),'USD')

Rata-rata Profit Margin [Region-1] --> 3961649.85 USD
Rata-rata Profit Margin [Region-2] --> 4560451.06 USD
Rata-rata Profit Margin [Region-3] --> 4044038.67 USD


setelah dilakukan metode `bootsrap` dapat dilihat bahwa terjadi peningkatan jumlah profit dan semakin positif. wilayah dengan profit tertinggi adalah wilayah `geo1` dengan profit `4.5` juta USD

In [35]:
# Menggabungkan hasil perhitungan bootstrapping
geo0_new = pd.Series(rev_geo0_boot, name='Region-1') / 1000000
geo1_new = pd.Series(rev_geo1_boot, name='Region-2') / 1000000
geo2_new = pd.Series(rev_geo2_boot, name='Region-3') / 1000000
combine = pd.concat([geo0_new, geo1_new, geo2_new], axis=1)

## Hitung Resiko dan keuntungan<a id='risk'></a>

Penting untuk mengetahui risiko dari masing-masing wilayah, kita akan memilih wilayah yang memiliki risiko paling rendah dengan ambang batas di bawah `2.5%` dan memilih rata-rata margin profit paling tinggi. Kita akan mencari selang kepercayaan berdasarkan tingkat keyakinan sebesar `95%`.

In [32]:
# selang kepercayaan 95%
cfd_interval = 0.95

# ambang batas kerugian 2.5%
risk_threshold = 0.25

In [33]:
# Fungsi untuk menghitung selang kepercayaan dan risiko
def cfd_risk(dif_rev, sample):
    low = dif_rev.quantile(0.025) * 1000000
    high = dif_rev.quantile(0.975) * 1000000
    negative = [neg for neg in dif_rev if neg < 0]
    risk = len(negative) / sample * 100
    return low, high, risk


In [36]:
# Menghitung selang kepercayaan dan risiko
low1, high1, risk1 = cfd_risk(geo0_new, n_boot)
low2, high2, risk2 = cfd_risk(geo1_new, n_boot)
low3, high3, risk3 = cfd_risk(geo2_new, n_boot)

# Menampilkan hasil selang kepercayaan dan risiko
print(f'Selang Kepercayaan [Region-1]: ({low1:.2f} sampai {high1:.2f}) USD')
print(f'Risiko [Region-1]: {risk1:.2f}%')
print(f'Selang Kepercayaan [Region-2]: ({low2:.2f} sampai {high2:.2f}) USD')
print(f'Risiko [Region-2]: {risk2:.2f}%')
print(f'Selang Kepercayaan [Region-3]: ({low3:.2f} sampai {high3:.2f}) USD')
print(f'Risiko [Region-3]: {risk3:.2f}%')

Selang Kepercayaan [Region-1]: (-1112155.46 sampai 9097669.42) USD
Risiko [Region-1]: 6.90%
Selang Kepercayaan [Region-2]: (338205.09 sampai 8522894.54) USD
Risiko [Region-2]: 1.50%
Selang Kepercayaan [Region-3]: (-1633504.13 sampai 9503595.75) USD
Risiko [Region-3]: 7.60%


selang kepercayaan pada wilayah `geo3` memiliki rentang paling tinggi dan resiko paling tinggi yaitu `7.60%` sedangkan paling rendah adalah wilayah `geo2` dengan resiko `1.5%`

## Kesimpulan<a id='conclusion'></a>

1. Rata-rata volume minyak pada wilayah `geo0` adalah `92.59` ribu barel dengan `RSME` `37.58` ribu barel
2. Rata-rata volume minyak pada wilayah `geo1` adalah `68.73` ribu barel dengan `RSME` `0.89` ribu barel. wilayah ini memberikan nilai `RSME` terbaik dan terendah dibandingkan dengan wilayah lain.
3. Rata-rata volume minyak pada wilayah `geo2` adalah `94.97` ribu barel dengan `RSME` `40.03` ribu barel. ini adalah wilayah  dengan rata-rata volume minyak tertinggi dibandingkan dengan wilayah lainnya. akan tetapi nilai `RSME` nya juga paling tinggi.
4. terlihat bahwa semuanya tidak memenuhi ambang batas produksi minimal yaitu `111.11` unit, dan yang paling mendekati adalah wilayah `geo2`
5. Ternyata ketika kita mengambil 200 lokasi terbaik dari dataset kita tidak mengalami kerugian, bahkan menunjukan keuntungan (nilai positif margin). 
6. setelah dilakukan metode `bootsrap` dapat dilihat bahwa terjadi peningkatan jumlah profit dan semakin positif. wilayah dengan profit tertinggi adalah wilayah `geo1` dengan profit `4.5` juta USD
7. selang kepercayaan pada wilayah `geo3` memiliki rentang paling tinggi dan resiko paling tinggi yaitu `7.60%` sedangkan paling rendah adalah wilayah `geo2` dengan resiko `1.5%`


### Temuan<a id='end'></a>

1. ketiga dataset terdiri dari 5 kolom, `id`, `f0`, `f1`, `f2`, dan `product` dan 100000 data, tipe data sudah benar, tidak ada nilai hilang dan tidak ada duplikasi data. jadi tidak perlu penanganan lanjutan dan bisa langsung ke proses selanjutnya.
2. dataset akan dipisahkan menjadi `75:25` antara `training set` dan `validation set`
3. semua dataset tidak memenuhi ambang batas produksi minimal
4. untuk mengetahui tingkat resiko an selang kepercayaan diperlukan teknik `bootstrap`

### Rekomendasi<a id='recom'></a>

Wilayah `geo1` adalah pilihan terbaik untuk pengembangan 200 sumur minyak baru berdasarkan analisis keuntungan, risiko, dan akurasi prediksi. Dengan keuntungan tertinggi dan risiko kerugian yang terkelola, wilayah ini memberikan potensi terbaik untuk investasi OilyGiant. Namun, wilayah `geo2` tetap menjadi alternatif yang layak dipertimbangkan mengingat volume minyak yang lebih tinggi dan risiko yang rendah. Hindari pengembangan di wilayah `geo0` pada tahap awal karena performa yang kurang optimal dibandingkan dua wilayah lainnya.

[Kembali](#back)